# Week 10 Homework - Kendall Gilbert

>Parks and green spaces provide a variety of health benefits by promoting physical activity, connection to nature and opportunities for community engagement. Nature-based programs can even be prescribed by health care providers as part of alternative, cost-effective treatment plans.

>In a paper recently published in the North Carolina Medical Journal, Lincoln Larson and Aaron Hipp, associate professors of parks, recreation and tourism management at NC State, provide an overview of how parks and green spaces can help alleviate contemporary health challenges.

[Source](https://cnr.ncsu.edu/news/2022/04/parks-green-spaces-improve-health/)

## 10.1 Finding the nearest parks to schools in Los Angeles County
You are given information on the point locations of schools in the LA Unified Schoo School District (LAUSD) (``` data/Schools_LAUSD.geojson```) and polygons of LA city parks (```data/Recreation_and_Parks.geojson ```)

1. Load the school and park data. For calcuating distances, what should you check and potentially change? 

In [14]:
#Import Modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

In [15]:
#Load data
LAUSD_Schools_Data = gpd.read_file('data/Schools_LAUSD.geojson')
Parks_Data = gpd.read_file('data/Recreation_and_Parks.geojson')

You should check and potentially change the crs of the data to calculate the distances.

In [16]:
#Check crs
print(LAUSD_Schools_Data.crs)
print(Parks_Data.crs)

epsg:4326
epsg:4326


The crs are the same, but they need to be changed to a projected coordinate system.

In [17]:
#Change to projected crs
LAUSD_Schools_Data = LAUSD_Schools_Data.to_crs(epsg=3857)
Parks_Data = Parks_Data.to_crs(epsg=3857)

2. Find the nearest park to each school in the LAUSD. Capture the  ``` Name ``` of the closest park. Note, this is not as straight forward as just copying and pasting from section-02 week 10! You actually have to think logically what columns are needed (centroids or geometries? what is the target dataframe?). Think carefully!

In [22]:
Parks_Data['Centroid'] = Parks_Data['geometry'].centroid

In [20]:
#Define the get_nearest_values function
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["Centroid"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["Centroid"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [21]:
#Add a column to the schools dataframe with the name of the nearest park
LAUSD_Schools_Data["Nearest_Park"] = LAUSD_Schools_Data.apply(get_nearest_values,other_gdf=Parks_Data,point_column="geometry",value_column='Name',axis=1)
LAUSD_Schools_Data.head()

,ID,LOCN,MPD_NAME,ADDRESS,CITY,ZIP,MPD_TYPE,MAP_TYPE,LABEL,MAP_DESC,MPD_DESC,LD,FULLNAME,CDSCODE,CHARTER,MAP_TYPE_MAPPING,TOOLTIP,NLA_URL,geometry,Nearest_Park
0,1,2835,VISTA DEL VALLE AC DL,12411 N BROMONT AVE,LOS ANGELES,91340,DS,ED,Vista del Valle Ac DL,Spanish Dual Language Program,Dual Language Program,0,Vista del Valle Dual Language Academy Dual Lan...,,,ES,Name: Vista del Valle Dual Language Academy Du...,navigatela/reports/lausd_school_report.cfm?PK=1,POINT (-13182641.899 4068553.705),Hubert H. Humphrey Memorial Park
1,2,2820,PARMELEE AVE EL DL SP,1338 E 76TH PL,LOS ANGELES,90001,DS,ED,Parmelee Ave El DL Sp,Spanish Dual Language Program,Dual Language Program,0,Parmelee Avenue Elementary Dual Language Spanish,,,ES,Name: Parmelee Avenue Elementary Dual Language...,navigatela/reports/lausd_school_report.cfm?PK=2,POINT (-13163640.151 4024782.311),Fremont High School Pool
2,3,2773,OCHOA LC DLC SP,5057 LIVE OAK ST,CUDAHY,90201,DS,ED,Ochoa LC DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Ellen Ochoa Learning Center Dual Language Spanish,,,ES,Name: Ellen Ochoa Learning Center Dual Languag...,navigatela/reports/lausd_school_report.cfm?PK=3,POINT (-13155359.027 4024496.783),Jordan Downs Recreation Center
3,4,2840,WILTON PL EL DLC KO,745 S WILTON PL,LOS ANGELES,90005,DK,ED,Wilton Pl El DLC Ko,Korean Dual Language Program,Dual Language Program,0,Wilton Place Elementary Dual Language Korean,,,ES,Name: Wilton Place Elementary Dual Language Ko...,navigatela/reports/lausd_school_report.cfm?PK=4,POINT (-13170806.628 4036664.178),Harold A. Henry Park
4,5,2814,MILES AVE EL DLC SP,6020 MILES AVE,HUNTINGTON PARK,90255,DS,ED,Miles Ave El DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Miles Avenue Elementary Dual Language Spanish,,,ES,Name: Miles Avenue Elementary Dual Language Sp...,navigatela/reports/lausd_school_report.cfm?PK=5,POINT (-13159944.823 4025787.724),Pueblo del Rio Recreation Center


3. Create an interactive map which plots the LAUSD schools locations and has a ``` Tooltip ``` capability of showing the ``` FULLNAME ``` of the school (using alias of ``` School Name ```) and the name of the closest park using the alias of ``` Nearest Park Name ```.

In [42]:
import folium

In [43]:
# Convert points to GeoJSON
School_Points_Gjson = folium.features.GeoJson(LAUSD_Schools_Data, name="School Name")

In [46]:
Schools_Map = folium.Map(location=[34, -118], tiles = 'cartodbpositron', zoom_start=10, control_scale=True)
School_Points_Gjson.add_to(Schools_Map)
folium.features.GeoJson(LAUSD_Schools_Data,  
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['FULLNAME','Nearest_Park'],
                                                                aliases = ['School Name','Nearest Park Name'],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(Schools_Map)
Schools_Map

4. Save your map as a .html file, upload to your GitHub, and provide a link where anyone can see the final product. Paste this link into the notebook you submit for the assignment. 

In [47]:
outfp = "LAUSD_Schools_and_Nearest_Parks_Map.html"
Schools_Map.save(outfp)

Here is the link to the map: https://kcpgilbert.github.io/Geospatial-Analysis-Spring-2023/LAUSD_Schools_and_Nearest_Parks_Map.html